<a href="https://colab.research.google.com/github/DevanMayer/EE-480-Assignments/blob/main/CA6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **EE 480 - Coding Assignment #6: Optimal Power Flow & Market Simulation**

**Instructor:** Dr. Saeed Manshadi

**Due Date:** Nov. 17, 2025

### **Instructions**

This assignment is the culmination of our work on Optimal Power Flow (OPF) and its application in electricity markets. You will compare different OPF formulations and participate in a simulated market to develop a profit-maximizing bidding strategy. All solutions must be calculated using **Julia** and the `PowerModels.jl` package.

**Working in Google Colab:**
*   This notebook is designed to be completed in Google Colab.
*   **CRITICAL FIRST STEP:** Ensure the Julia kernel is active. Run the setup cell below.

---
### **Required Packages (Run This Cell First!)**

In [6]:
# This cell adds the necessary packages for this assignment.
using Pkg
Pkg.add("PowerModels")
Pkg.add("Ipopt")
Pkg.add("JSON")
Pkg.add("LinearAlgebra")
Pkg.add("Printf") # For formatted printing
Pkg.add("PowerModels")
Pkg.add("HiGHS")
Pkg.add("JuMP")


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.to

---
### **Problem 1: Comparing DC OPF and AC OPF**

The DC OPF is a linear approximation, while the AC OPF is a full, nonlinear model. In this problem, you will solve both for the standard IEEE 14-bus system and analyze the differences in the results.

**Guidance:**
*   Load the `case14.m` file, which includes default generator cost data.
*   Run the `solve_dc_opf` function to get the DC solution.
*   Run the `solve_ac_opf` function to get the AC solution.
*   Present your results in a clean, comparative table.

In [8]:
using PowerModels, Ipopt, Printf, LinearAlgebra, JSON

# --- Step 1: Load the test case ---
case_file_path = joinpath(dirname(pathof(PowerModels)), "..", "test/data/matpower/case14.m")
case_14bus = PowerModels.parse_file(case_file_path)

# --- Step 2: Solve the DC OPF ---
result_dc = solve_opf(case_14bus, DCMPPowerModel, Ipopt.Optimizer)


# --- Step 3: Solve the AC OPF ---
result_ac = solve_opf(case_14bus, ACPPowerModel, Ipopt.Optimizer)


# --- Step 4: Compare the Results ---
println("--- OPF Solution Comparison: DC vs. AC ---")
println("Total DC OPF Cost: ", round(result_dc["objective"], digits=2), "/hr")
println("Total AC OPF Cost: ", round(result_ac["objective"], digits=2), "/hr")

println("\n" * "="^60)
println("           GENERATOR DISPATCH COMPARISON (MW)")
println("="^60)
println("Gen ID | Bus | DC Dispatch (MW) | AC Dispatch (MW) | Difference (MW)")
println("-----------------------------------------------------------------")

for (i, gen_dc) in result_dc["solution"]["gen"]
    gen_ac = result_ac["solution"]["gen"][i]
    pg_dc = gen_dc["pg"] * 100
    pg_ac = gen_ac["pg"] * 100
    diff = pg_ac - pg_dc
    bus_num = case_14bus["gen"][i]["gen_bus"] # Corrected line
    @printf "   %-3s | %3d | %16.2f | %16.2f | %15.2f\n" i bus_num pg_dc pg_ac diff
end

println("\nQuestion for your report: Why is the total cost of the AC OPF higher than the DC OPF? (Hint: Think about losses).")

[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 4 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 4 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 1 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 1 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 12 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 12 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 20 from -360.0 to -60.0 deg.
[w



---
### **Problem 2: From Economic Dispatch to Security-Constrained OPF**

This problem demonstrates the evolution of dispatch methods. We will use the 3-bus system from our lecture.

*   **System Data:** 3 buses, 2 generators, 1 load.
    *   Gen 1 (Bus 1): Cost = `10*P`, Pmax = 5.0 p.u.
    *   Gen 2 (Bus 3): Cost = `25*P`, Pmax = 5.0 p.u.
    *   Load (Bus 2): `P_D = 2.0` p.u.
    *   Line reactances: `x₁₂ = 0.1`, `x₁₃ = 0.4`, `x₂₃ = 0.2`.
    *   Line thermal limit: `P_flow_max = 1.2` p.u. for the line between Bus 2 and 3.

(a) **Economic Dispatch:** Ignoring all transmission constraints, what is the cheapest way to serve the 2.0 p.u. load? What is the total cost?

(b) **DC Power Flow Check:** For the dispatch from part (a), calculate the DC power flow on the line between Bus 2 and 3. Is the dispatch feasible (i.e., is the flow below the thermal limit)?

(c) **DC OPF:** Using `PowerModels.jl`, solve the DC OPF for this system. What is the new, secure dispatch and the new total cost? What is the "cost of congestion"?

In [19]:
# --- Problem 2 Solution ---

# --- System Data ---
P_D2 = 2.0
Cost_G1 = 10.0; Cost_G2 = 25.0;
Pmax_G1 = 5.0; Pmax_G2 = 5.0;
x12 = 0.1; x13 = 0.4; x23 = 0.2;
P_flow_max_23 = 1.2;

# --- Part (a): Economic Dispatch ---
# To serve 2.0 p.u. of load, the cheapest method is to use the cheapest generator.
P_G1_ed = P_D2;
P_G2_ed = 0;
total_cost_ed = P_G1_ed*Cost_G1+P_G2_ed*Cost_G2;
println("--- Part (a): Economic Dispatch ---")
@printf "Dispatch: P_G1 = %.1f p.u., P_G2 = %.1f p.u.\n" P_G1_ed P_G2_ed
@printf "Total Cost = \$%.1f/hr\n" total_cost_ed

# --- Part (b): DC Power Flow Check ---
# We need to find the angles that result from this dispatch.
# P_inj = B' * δ => δ = inv(B') * P_inj
b12 = 1/x12; b13 = 1/x13; b23 = 1/x23;
B_prime = [b12+b23  -b23; -b23  b13+b23]
# Injections at non-slack buses [2, 3]
P_inj = [-P_D2, P_G2_ed]
delta_vec = inv(B_prime) * P_inj
delta2 = delta_vec[1]; delta3 = delta_vec[2];

# Now calculate the flow on line 2-3
P_flow_23 = (delta2 - delta3) / x23
is_feasible = abs(P_flow_23) <= P_flow_max_23

println("\n--- Part (b): Feasibility Check ---")
@printf "Flow on Line 2-3 is %.3f p.u. (Limit is %.1f p.u.)\n" P_flow_23 P_flow_max_23
println("Is the dispatch feasible? ", is_feasible)

# --- Part (c): DC OPF Solution ---
# Create the PowerModels case dictionary for this problem
case_p2 = Dict("baseMVA" => 100.0, #data organized from ChatGPT
    "bus" => Dict(
        "1" => Dict("bus_i"=>1, "bus_type"=>3, "pd"=>0.0,   "qd"=>0.0, "vm"=>1.0, "va"=>0.0),
        "2" => Dict("bus_i"=>2, "bus_type"=>1, "pd"=>200.0, "qd"=>0.0, "vm"=>1.0, "va"=>0.0),
        "3" => Dict("bus_i"=>3, "bus_type"=>2, "pd"=>0.0,   "qd"=>0.0, "vm"=>1.0, "va"=>0.0)
    ),
    "gen" => Dict(
        "1" => Dict("gen_bus"=>1, "pg"=>0.0, "qg"=>0.0, "qmax"=>0.0, "qmin"=>0.0,
                    "pmax"=>500.0, "pmin"=>0.0, "cost"=>[0.0, 10.0]),
        "2" => Dict("gen_bus"=>3, "pg"=>0.0, "qg"=>0.0, "qmax"=>0.0, "qmin"=>0.0,
                    "pmax"=>500.0, "pmin"=>0.0, "cost"=>[0.0, 25.0])
    ),
    "branch" => Dict(
        "1" => Dict("f_bus"=>1, "t_bus"=>2, "br_r"=>0.0, "br_x"=>0.1, "rateA"=>9999.0),
        "2" => Dict("f_bus"=>1, "t_bus"=>3, "br_r"=>0.0, "br_x"=>0.4, "rateA"=>9999.0),
        "3" => Dict("f_bus"=>2, "t_bus"=>3, "br_r"=>0.0, "br_x"=>0.2, "rateA"=>120.0)
    ))
# ... (refer to lecture notes for the structure)

# Solve the DC OPF
result_dc_opf = solve_dc_opf(case_p2, Ipopt.Optimizer)
P_G1_opf = result_dc_opf["solution"]["gen"]["1"]["pg"]
P_G2_opf = result_dc_opf["solution"]["gen"]["2"]["pg"]
total_cost_opf = result_dc_opf["objective"]
cost_of_congestion = total_cost_opf - total_cost_ed

println("\n--- Part (c): DC OPF Secure Dispatch ---")
@printf "Secure Dispatch: P_G1 = %.2f p.u., P_G2 = %.2f p.u.\n" P_G1_opf P_G2_opf
@printf "New Total Cost = \$%.2f/hr\n" total_cost_opf
@printf "The Cost of Congestion is \$%.2f/hr\n" cost_of_congestion

--- Part (a): Economic Dispatch ---
Dispatch: P_G1 = 2.0 p.u., P_G2 = 0.0 p.u.
Total Cost = $20.0/hr

--- Part (b): Feasibility Check ---
Flow on Line 2-3 is -0.286 p.u. (Limit is 1.2 p.u.)
Is the dispatch feasible? true


LoadError: KeyError: key :load not found



---
### **Problem 3: The Bidding Game**

You are the owner of the generator at **Bus 11** in the IEEE 30-bus system. Your goal is to maximize your profit. We will use the same congested market case from the lecture, where the line between Bus 2 and Bus 4 (Branch 4) is derated.

(a) **Marginal Bidding:** First, let's assume you and all other generators bid at your "marginal cost" (the costs we added in the lecture). Run the AC OPF and determine the dispatch, the LMP at your bus, and your resulting profit (`Profit = Dispatch_MW * LMP`). This is your baseline.

(b) **Strategic Bidding:** Now, you will develop a new, strategic bid for your generator at Bus 11. Assume all other generators continue to bid their marginal cost. Your goal is to change your bid to increase your profit. Can you exercise market power? Try submitting a higher bid. Does your dispatch go down? Does your profit go up? Experiment and find a bid that improves your profit over the baseline.

**Guidance:**
*   You will start with the `market_case` from the lecture.
*   For part (a), you will run the AC OPF with the default linear costs.
*   For part (b), you will create a `deepcopy` of the market case and **only modify the `gencost` entry for your generator** (Gen ID "5" is at Bus 11). Then re-run the OPF and calculate your new profit.


In [47]:
# --- Problem 3 Solution ---
Pkg.add("HiGHS")
using PowerModels, HiGHS, LinearAlgebra, Printf
using PowerModels, Ipopt, Printf

result_market = PowerModels.solve_dc_opf(
    market_case,
    HiGHS.Optimizer;
    setting = Dict("output" => Dict("duals" => true))
)

# --- Setup: Recreate the Congested Market Case from the Lecture ---
# ... (Copy the code from the lecture that loads case30, adds default costs, and derates Branch 4) ...
case_file_path = joinpath(dirname(pathof(PowerModels)), "..", "test/data/matpower/case30.m")
case_data = PowerModels.parse_file(case_file_path)

# --- Part (a): Baseline Profit with Marginal Bidding ---
# Solve the AC OPF with the default costs and enable duals
result_base = solve_ac_opf(
    market_case,
    Ipopt.Optimizer;
    setting = Dict("output" => Dict("duals" => true))
)

# Extract LMPs (active power duals)
lmps = Dict(parse(Int, b) => bus["lam_kcl_r"]
            for (b, bus) in result_base["solution"]["bus"])

update_data!(market_case, result_base["solution"])

println("\n--- Part (a): Baseline Profit ---")
team_num = 1
bus_num = 11
gen_id = "5"

# Generator dispatch
gen_sol = result_base["solution"]["gen"][gen_id]
dispatch_mw = gen_sol["pg"] * market_case["baseMVA"]

# LMP at generator bus
lmp = lmps[bus_num]

profit = dispatch_mw * lmp

@printf " %-4s| %6s | %3d | %13.1f | %12.2f | %12.2f\n" team_num 1 bus_num dispatch_mw lmp profit

# --- Part (b): Strategic Bidding ---
# Create a copy of the market case to modify your bid
case_strategic = deepcopy(market_case)

# Define your new, strategic bid for Gen 5 (at Bus 11)
my_strategic_bid = [60.0, 0.0]
case_strategic["gen"]["5"]["cost"] = my_strategic_bid

# Solve the OPF with your new bid and enable duals
result_strategic = solve_ac_opf(
    case_strategic,
    Ipopt.Optimizer;
    setting = Dict("output" => Dict("duals" => true))
)

# Extract new LMPs
lmps_strategic = Dict(parse(Int,b) => bus["lam_kcl_r"]
                      for (b,bus) in result_strategic["solution"]["bus"])

# Generator dispatch and profit
gen_sol_new = result_strategic["solution"]["gen"]["5"]
dispatch_mw_new = gen_sol_new["pg"] * case_strategic["baseMVA"]
lmp_new = lmps_strategic[bus_num]
profit_new = dispatch_mw_new * lmp_new

println("\n--- Part (b): Strategic Bidding Results ---")
@printf " %-4s| %6s | %3d | %13.1f | %12.2f | %12.2f\n" team_num 1 bus_num dispatch_mw_new lmp_new profit_new

println("\nCompare your new profit to the baseline. Did your strategy work?")


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 113 rows; 77 cols; 294 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 3e+01]
  Cost    [5e+01, 1e+02]
  Bound   [2e-01, 8e+00]
  RHS     [2e-02, 9e-01]
Solving LP with useful basis so presolve not used
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -7.6278980447e-05 Pr: 46(157.201); Du: 12(4.51728e-05) 0s
         35     1.8870387914e+02 Pr: 0(0) 0s

Model status        : Optimal
Simplex   iterations: 35
Objective value     :  1.8870387914e+02
P-D objective error :  1.5021737165e-16
HiGHS run time      :          0.00
[info | PowerModels]: removing 3 cost terms from generator 4: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 1: [52.1378, 0.0]
[info | PowerModels]: removing 3 cost terms from generator 5: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 2: [113.51659999999998, 0.0]
[